In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [97]:
def scrape_wiki(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
    else:
        print("Error fetching the webpage. Status code:", response.status_code)
    table_data = []
    first_tag = soup.find('h3')
    state = first_tag.text
    state = state.replace('[edit]','')
 
    table = soup.find('table',class_='wikitable collapsible collapsed')  
    table_list = load_table(table, 'state', state)
    poll_count = 0
    for row in table_list:
        if poll_count <= 15:
            table_data.append(row)
            poll_count += 1

    
    table_header = ['Poll Source', 'Date administered (2008)', 'Barack Obama', 'John McCain', 'Margin', 'Sample size', 'Margin of error', 'state']
    
    headers = first_tag.find_all_next('h3')
    for header in headers:
        state = header.text
        state = state.replace('[edit]','') 
        next_table = header.find_next_sibling('table', class_='wikitable collapsible collapsed')
        if next_table:
            t_date  = load_table(next_table, 'state', state)
            poll_count = 0
            for t in t_date:
                if t != table_header:
                    if poll_count <= 15: 
                        table_data.append(t)
                        poll_count += 1
                    
    return table_data




        
def load_table(table, descriptor_title = 'none', descriptor = 'none'):
    t = table
    table_data = []
    row_data = []
    rows = t.find_all('tr')
    heads = t.find_all('th')
    heads = [head.text.strip() for head in heads]
    if descriptor_title != 'none':
        heads.append(descriptor_title)
    
    table_data.append(heads)

    # print(heads)

    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        if descriptor != 'none':
            cols.append(descriptor)
        if len(cols) > 1:
            table_data.append(cols)         
    return table_data


In [98]:
url = 'https://en.wikipedia.org/wiki/Statewide_opinion_polling_for_the_2008_United_States_presidential_election'
polls = scrape_wiki(url)

In [99]:
polling = pd.DataFrame(polls)

In [100]:
polling.head()
polling.columns = polling.iloc[0]
polling = polling[1:]

In [103]:
polling.loc[polling.state == 'Michigan']

,Poll Source,Date administered (2008),Barack Obama,John McCain,Margin,Sample size,Margin of error,state
286,Mitchell Research & Communications Inc.,November 3,54%,38%,16,400 LV,±5%,Michigan
287,Polimetrix/YouGov,October 18–November 1,55%,42%,13,973 RV,Not reported,Michigan
288,Detroit Free Press-Local 4/Selzer & Co.[perman...,October 28–31,53%,37%,16,616 LV,±4%,Michigan
289,Public Policy Polling,October 28–30,55%,42%,13,"1,532 LV",±2.5%,Michigan
290,Mitchell Research & Communications Inc.,October 26–30,54%,40%,14,600 LV,±4%,Michigan
291,Rasmussen Reports/Pulse Opinion Research,October 28,53%,43%,10,500 LV,±4.5%,Michigan
292,Detroit News/EPIC-MRA[permanent dead link],October 26–28,50%,38%,12,400 LV,±4.9%,Michigan
293,Big Ten,October 19–22,58.1%,35.8%,22.3,562 LV,±4.2%,Michigan
294,Washington Post/Wall Street Journal/Quinnipiac...,October 8–12,54%,38%,16,"1,043 LV",±3%,Michigan
295,Rasmussen Reports/Pulse Opinion Research,October 8,56%,40%,16,500 LV,±4.5%,Michigan
